# TRAVEL BOT using DEEP LEARNING
BY: SINY P RAPHEL

### IMPORT NECESSARY PACKAGES

In [1]:
import pandas as pd
import numpy as np
import os
import json

# import nltk
# import re
# #nltk.download('stopwords')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

#!pip install colorama
import colorama 
colorama.init()
from colorama import Fore, Style, Back
import random

### DATA LOAD
The data is stored as a json file in the same folder in the drive. json.load() used to extract data from the file.

In [2]:
data_dir = "drive/MyDrive/NLP_Project/"

In [3]:
with open(data_dir+'data.json') as file:
    data = json.load(file)

### DATA PREPROCESSING
Each feature is extracted from the dictionary and saved into lists.

In [4]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

The target is encoded using the LabelEncoder()

In [5]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

### DATA FEATURE EXTRACTION
The data is 
* converted to lower case
* removed punctuations
* out-of-vocabulary words handled
* converted to tokens and padded

In [6]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

#training_sentences = normalize_corpus(training_sentences)
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token, lower=True)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

### MODEL TRAINING
Sequential() class of Keras package is used for training the data. The model consists of one input layer, two hidden layers and one output layer.
The loss is set to sparse crossentropy and the performance evaluated using "accuracy"

In [7]:

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 9)                 153       
                                                                 
Total params: 16,697
Trainable params: 16,697
Non-trainable params: 0
____________________________________________________

### MODEL EVALUATION
Epoch is set to 500. To limit the epoch to be printed a print_epoch variable is assigned and a call back function is developed.

In [8]:
epochs = 500
print_epoch = 100

In [9]:
class Callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
      # if epoch count is a multiple of assigned print_epoch value, print the values
      if epoch % print_epoch == 0:
        print(f"Epoch {epoch} : Loss is {logs['loss']} and Accuracy is {logs['accuracy']}")
      elif epoch == epochs-1:
        print(f"Epoch {epoch+1} : Loss is {logs['loss']} and Accuracy is {logs['accuracy']}")

In [10]:
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs, verbose=0, callbacks=[Callback()])

Epoch 0 : Loss is 2.1964285373687744 and Accuracy is 0.1666666716337204
Epoch 100 : Loss is 1.9513945579528809 and Accuracy is 0.3611111044883728
Epoch 200 : Loss is 1.2899298667907715 and Accuracy is 0.4444444477558136
Epoch 300 : Loss is 0.8568596243858337 and Accuracy is 0.8055555820465088
Epoch 400 : Loss is 0.5128582715988159 and Accuracy is 0.9166666865348816
Epoch 500 : Loss is 0.3006773293018341 and Accuracy is 0.9444444179534912


Accuracy is always 1 and there is no point in considering the accuracy. We can see that the loss reduces as the epochs increases.

### Chatbot
The function chat() runs utilizes the trained model and initializes the chatbot

In [12]:
def chat():
    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: hello
ChatBot: Hi
User: whats your name
ChatBot: Just call me as TravBot
User: hellonTrav
ChatBot: Tell me how can assist you
User: hello trav
ChatBot: Hi
User: sign up for airline perk card
ChatBot: Yes you can, I can help you to create one
User: thank you
ChatBot: My pleasure
User: quit
